## Creating a mesh

We can also use the Splatter wrapper class to take an existing nerfstudio model and create a mesh!
1. **mesh:** creates a mesh via TSDF fusion

2. **query_mesh:** uses the trained model to query the mesh and returns a similarity map

3. **plot_mesh:** enables plotting of mesh features



In [1]:
import os, sys
from pathlib import Path
from ns_extension.wrapper import Splatter, SplatterConfig

import pyvista as pv
pv.start_xvfb()

# import vtkmodules.all as vtk
# render_window = vtk.vtkRenderWindow()
# render_window.SetOffScreenRendering(1)


[Taichi] version 1.7.3, llvm 15.0.4, commit 5ec301be, linux, python 3.10.18


[I 07/18/25 15:10:54.859 15710] [shell.py:_shell_pop_print@23] Graphical python shell detected, using wrapped sys.stdout


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


/opt/conda/envs/nerfstudio/lib/python3.10/site-packages/pyvista/plotting/utilities/xvfb.py:48: PyVistaDeprecationWarning: This function is deprecated and will be removed in future version of PyVista. Use vtk-osmesa instead.
  warnings.warn(


Set paths to the file for running splats

In [2]:
base_dir = Path('/workspace/fieldwork-data/')
session_dir = base_dir / "rats/2024-07-11/SplatsSD"

# Make the configuration 
splatter_config = SplatterConfig(
    file_path=session_dir / "C0119.MP4",
    method='rade-features',
    frame_proportion=0.25, # Use 25% of the frames within the video (or default to minimum 300 frames)
)

# Initialize the Splatter class
splatter = Splatter(splatter_config)

# Call these to populate the splatter with paths (probably a better way to do this --> maybe save out config)
splatter.preprocess()
splatter.extract_features()

transforms.json already exists at /workspace/fieldwork-data/rats/2024-07-11/environment/C0119/preproc/transforms.json
To rerun preprocessing, set overwrite=True
Output already exists for rade-features
To rerun feature extraction, set overwrite=True


### Create a mesh

We can create a mesh by calling the ```mesh()``` method. Under the hood, this runs TSDF fusion creating an integrated volume. 

In [3]:
splatter.mesh(overwrite=True)


Available runs:
[0] 2025-07-11_171420


In [8]:
similarity = splatter.query_mesh(
    positive_queries=["tree"],
    negative_queries=["ground", "leaves"]
)

Loading model from /workspace/fieldwork-data/rats/2024-07-11/environment/C0119/rade-features/2025-07-11_171420/config.yml


[14:20:50] Auto image downscale factor of 2                                                 ]8;id=940919;file:///opt/conda/envs/nerfstudio/lib/python3.10/site-packages/nerfstudio/data/dataparsers/nerfstudio_dataparser.py\nerfstudio_dataparser.py]8;;\:]8;id=733151;file:///opt/conda/envs/nerfstudio/lib/python3.10/site-packages/nerfstudio/data/dataparsers/nerfstudio_dataparser.py#484\484]8;;\

[14:21:18] use color only optimization with sigmoid activation                                         ]8;id=274394;file:///opt/conda/envs/nerfstudio/lib/python3.10/site-packages/nerfstudio/models/splatfacto.py\splatfacto.py]8;;\:]8;id=97910;file:///opt/conda/envs/nerfstudio/lib/python3.10/site-packages/nerfstudio/models/splatfacto.py#266\266]8;;\

Loading latest checkpoint from load_dir

✅ Done loading checkpoint from 
/workspace/fieldwork-data/rats/2024-07-11/environment/C0119/rade-features/2025-07-11_171420/nerfstudio_models/step-00002
9999.ckpt

In [5]:
splatter.plot_mesh()

Number of points: 133390
Number of cells: 247846
Bounds: BoundsTuple(x_min=-0.8834666609764099, x_max=0.7600077986717224, y_min=-0.12439944595098495, y_max=2.6558642387390137, z_min=-1.2267214059829712, z_max=1.1309670209884644)


Widget(value='<iframe src="http://localhost:42195/index.html?ui=P_0x73e081502440_1&reconnect=auto" class="pyvi…

Plot similarity maps

In [7]:
splatter.plot_mesh(attribute=similarity)

Number of points: 137342
Number of cells: 253479
Bounds: BoundsTuple(x_min=-5.775000095367432, x_max=0.7600077986717224, y_min=-0.12439944595098495, y_max=7.386317729949951, z_min=-4.992317199707031, z_max=6.735000133514404)


Widget(value='<iframe src="http://localhost:39649/index.html?ui=P_0x7fe7a22b3250_2&reconnect=auto" class="pyvi…

In [6]:
import pyvista as pv

# Load the PLY file
mesh = pv.read(splatter.config['mesh_info']['mesh'])